In [4]:
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

# Auswerten eines Schach-Endspiels

Zum Vergleich, dass die berechneten Züge der KI eine gute Lösung für das Gewinnen einer Endspielsituation sind, wird das Programm Stockfish verwendet. Es stellt eine Schach-Engine dar, die nach einem zuvor definierten Muster optimale Schachzüge berechnet.

## Importe

Für die Durchführung der Tests und der Auswertung eines Schach-Endspiels gegen eine andere KI werden die hier stehenden Bibliotheken benötigt:

* ``chess``: Die Python-Schach-Bibliothek, mit welcher Schachbretter dargestellt werden, Züge und Zustände ausgewertet etc.
* ``stockfish``: Bibliothek, welche eine Python Schnittstelle für die Stockfish-Enginge anbietet.
* ``clear_output`` aus der IPython-display-Library: Eine Funktion, die für die Fortschrittsanzeige von Berechnungen
verwendet wird.
* ``display`` aus der IPython-display-Library: Wird zum grafischen Anzeigen der Schachbretter verwendet.
* ``json``: Wird zum Speichern der Ergebnisse verwendet.
* ``time``: Wird zum Verzögern von Ausgaben verwendet.

In [5]:
import chess
import stockfish
from IPython.display import clear_output
from IPython.display import display
import json
import time

## Konfiguration

Zu Beginn müssen für den Vergleich unterschiedliche Variablen definiert werden. Diese beinhalten eine Fen, die für ein gegebenes Szenario überprüft werden soll, den Pfad zu der berechneten $S_n$ Sequenz und einen Pfad zu der Stockfish-Installation. Weiterhin wird eine globale Variable ``VERBOSE`` festgelegt, die ein Anzeigen von zusätzlichen Kommentaren ermöglicht.

In [6]:
FEN = "4k3/8/8/8/8/8/8/R3K3 w - - 0 1"
S_N_FILE = "S_n_seq_12_01.json"
STOCKFISH_PATH = "./stockfish_14.1/stockfish_14.1_win_x64_avx2.exe"
VERBOSE = False

STOCKFISH = stockfish.Stockfish(STOCKFISH_PATH)

Der Vergleich der Spielbretter findet in diesem Notebook anhand der FEN-Schreibweise statt. Diese wird in zweierlei Form verwendet, in ihrer gesamten und in einer gekürzten Schreibweise. Die gekürzte Schreibweise beinhaltet nur das momentane Spielfeld, aber auch die Farbe, die gerade am Zug ist. Zur Erstellung dieser Schreibweise wurde die Funktion ``get_board_and_turn`` geschrieben. Diese erhält als Parameter eine komplette ``fen`` und gibt in der FEN-Notation das Brett und die Farbe, die am Zug ist, zurück.

In [7]:
def get_board_and_turn(fen):
    split = fen.split()
    short_fen = str(split[0]) + " " + str(split[1])
    return short_fen

Für die Bestimmung der Züge der KI werden die $S_n$ Mengen verwendet, die mit Hilfe des Notebooks ``calculation.ipynb`` bestimmt werden. Diese wurden bereits in einer JSON-Datei serialisiert abgespeichert. Für dieses Notebook werden alle FENs gelesen und daraus Board-Objekte erstellt. Dies geschieht mit der Funktion ``load_s_n_sequence``. Diese erhält den Dateipfad (``filepath``) als Parameter, um aus der Datei die $S_n$ Mengen auslesen zu können.


In [8]:
def load_s_n_sequence(filepath):
    s_n_sequence_fen = []
    s_n_sequence_short_fen = []
    f = open(filepath, "r")
    tmp = json.loads(f.read())
    for item in tmp:
        tmp_fen = set()
        tmp_fen_short = set()
        for board in item:
            tmp_fen.add(board)
            tmp_fen_short.add(get_board_and_turn(board))
        s_n_sequence_fen.append(tmp_fen)
        s_n_sequence_short_fen.append(tmp_fen_short)
    f.close()
    return s_n_sequence_fen, s_n_sequence_short_fen

In [9]:
S_N_Sequence_fen, S_N_Sequence_fen_short = load_s_n_sequence(S_N_FILE)

## Die Züge für die KI ermitteln

Der erste Schritt, um einen Zug für die KI zu ermitteln, liegt im Finden der momentanen Spielsituation. Dies wird mit der Funktion ``find_board_in_sequence`` erreicht. Sie durchsucht eine gegebene $S_n$ Sequenz nach einem übergebenen Schachbrett. Die Parameter hierfür lauten wie folgt:

* ``fen``: Das Board (als FEN), welches gefunden werden soll.
* ``sequence_short``: Eine Liste von $S_n$ Sequenzen, in welchen das Board gesucht wird, in gekürzter FEN-Schreibweise.

Das Ergebnis der Funktion kann insgesamt zwei unterschiedliche Formen annehmen:
* Einen Integer, der den $S_n$ Index (z.B. $S_3$) darstellt.
* Der Wert -1. Dies drückt aus, dass das Board nicht gefunden wurde.

Das Ergebniss wird mithilfe von dem anschließenden Algorithmus bestimmt:
1. Über die $S_n$ Sequenzen iterieren.
2. Schauen, ob die FEN in der $S_N$ Menge gefunden werden kann.
    1. Wenn das Board existiert, N zurückgeben.
    2. Wenn das Board nicht existiert, weitersuchen.
3. Falls das Board nicht gefunden wird, -1 zurückgeben.

In [10]:
def find_fen_in_sequence(fen, sequence_short):
    short = get_board_and_turn(fen)
    for i in range(len(sequence_short)):
        if short in sequence_short[i]:
            return i
    return -1

Nachdem das Brett in einer $S_n$ Menge gefunden wurde, ist der nächste Schritt den passenden Zug für die KI zu bestimmen. Zu diesem Zweck wurde die Funktion ``find_next_move`` definiert. Diese berechnet für eine übergebene FEN den idealen Spielzug. Weiterhin wird aus Effizienzgründen schon die Position des nächsten Bretts in der $S_n$ Sequenz zurückgegeben. Für diese Berechnung benötigt ``find_next_move`` nachkommende Argumente:

* ``fen``: Die FEN, für welches der nächste Spielzug berechnet werden soll.
* ``s_index``: Das $n$ eines $S_n$, in welchem sich die ``fen`` befindet. Wird aus Effizienzgründen übergeben.
* ``s_n_sequence_short``: Die Liste mit allen $S_n$ in kürzer FEN-Schreibweise.

Als Ergebnis liefert die Funktion den nächsten Move oder den Wert -1 als Hinweis, dass kein Spielzug berechnet werden konnte.

Der Algorithmus dieser Funktion erstreckt sich folgender Art und Weise:
* Führe alle möglichen Moves durch, bis das Ergebnis-Board in $S_{m}$ mit $m < n$ gefunden wurde.
* Gebe den neuen ``s_index`` und den gefundenen ``move`` zurück.
* Sofern das Board nicht gefunden wird, gebe -1 zurück.

In [11]:
def find_next_move(fen, s_index, s_n_sequence_short):
    STOCKFISH.set_fen_position(fen)
    curr_board = chess.Board(fen)
    if curr_board.turn:
        if VERBOSE:
            print("---White:---")
            print("Starting in S" + str(s_index))
        for move in curr_board.legal_moves:
            curr_board.push(move)
            cur_fen = curr_board.fen()
            _tmp = find_fen_in_sequence(cur_fen, [s_n_sequence_short[s_index - 1]])
            s_index_tmp = s_index - 1
            if _tmp != -1:
                if VERBOSE:
                    print("    Move: " + str(move))
                    print("    S" + str(s_index_tmp))
                    print("Ended in S" + str(s_index))
                curr_board.pop()
                return s_index_tmp, move
            curr_board.pop()

        return -1, None
    else:
        if VERBOSE:
            print("---Black:---")
            print("Starting in S" + str(s_index))
        move = chess.Move.from_uci(STOCKFISH.get_best_move())
        curr_board.push(move)
        cur_fen = curr_board.fen()
        s_index = find_fen_in_sequence(cur_fen, s_n_sequence_short[:s_index])
        if VERBOSE:
            print("    Move: " + str(move))
            print("    S" + str(s_index))
            print("Ended in S" + str(s_index))
        curr_board.pop()
        return s_index, move

## Vergleich zwischen Stockfish und KI

Für den Vergleich werden die Anzahlen der Züge zwischen Stockfish und der selbstgeschriebenen KI miteinander verglichen.

Bei der Berechnung der Züge für die KI wird die Funktion ``calculate_all_moves`` verwendet. Diese berechnet in einer Schleife alle Moves ausgehend von einer FEN, bis die KI gewonnen hat. Die Berechnung verwendet hierzu folgende Parameter:

* ``fen``: Die FEN der Spielsituation, die berechnet werden soll.
* ``s_n_sequence_short``: Die Liste der $S_n$ in Kurzschreibweise, welche für die Figurenkonstellation berechnet wurde.

Das Ergebnis der Ausführung ist eine Liste aller Moves, welche für das Board berechnet wurde oder der Wert None als Hinweis, dass kein Sieg erzielt werden konnte.

Die Liste wurde durch den anschließenden Algorithmus bestimmt:
* Einordnung der übergebenen FEN in der $S_n$ Sequenz finden.
* Solange $n > 0$ in einer Schleife den nächsten Spielzug berechnen.
* Die Liste der errechneten Spielzüge zurückgeben.

In [12]:
def calculate_all_moves(fen, s_n_sequence_short):
    moves = []

    s_index = find_fen_in_sequence(fen, s_n_sequence_short)

    board = chess.Board(fen)
    while s_index > 0:
        cur_fen = board.fen()
        s_index, next_move = find_next_move(cur_fen, s_index, s_n_sequence_short)
        board.push(next_move)
        moves.append(next_move)

    if s_index == -1:
        return None

    return moves

Nachdem eine Liste aller Züge für die KI bestimmt wurden, gilt dies gleichermaßen für die Berechnung von Stockfish umzusetzen. Hierfür erhält die Funktion ``stockfish_movelist`` den Parameter:
* ``fen``: Die FEN des Boardes, für welches alle Züge berechnet werden sollen.

Nach der Ausführung erhält man ebenso eine List von Spielzügen.

Die Liste wurde mit einem leicht abgeänderten Algorithmus bestimmt:
* Solange das Spiel nicht beendet ist, einen weiteren Zug berechnen.
* Am Ende alle Züge zurückgeben.

In [13]:
def stockfish_movelist(fen):
    moves = []

    board = chess.Board(fen)

    while not board.is_game_over():
        STOCKFISH.set_fen_position(board.fen())
        next_move = chess.Move.from_uci(STOCKFISH.get_best_move())
        board.push(next_move)
        moves.append(next_move)

    return moves


Bei `compare_sequence_stockfish` wird für jedes Board einer $S_n$ Menge überprüft, ob Stockfish oder die zuvor definierten Mengen das Board effizienter lösen. Dabei gilt es für den Vergleich zu beachten, dass folgende Vorgehensweise verwendet wird.
- `Stockfish`: Für sowohl schwarz, als auch für weiß, werden die Züge mit der Funktion ``Stockfish().get_best_move()`` bestimmt.
- `KI`: Für weiß werden die Züge anhand der $S_n$ Mengen bestimmt, die schwarzen Züge werden mit der Funktion ``Stockfish().get_best_move`` gewählt.

Die Funktion, die für den Vergleich mit Stockfish angewandt wurde, birgt die Gefahr, dass durch Stockfish für schwarz
nicht der beste Zug ausgewählt wird und dadurch das Ergebnis beeinflusst. Diese Gefahr erhöht sich vor allem dann, wenn
sich die Züge bis zu einem Schachmatt erhöhen, da bei jedem Zug von schwarz die Gefahr besteht, dass nicht der beste Zug
ausgewählt wird. Deshalb werden bei beiden Berechnungen die Züge für schwarz von Stockfish bestimmt, wobei das Risiko in geringem Maße bestehen bleibt.

Die Funktion erhält als Parameter:
- ``sequence_index``: Das N der Menge, für die alle Bretter überprüft werden sollen.
(Bsp.: S_10 -> ``sequence_index = 10``)
- ``s_n_sequence_fen``: Alle $S_n$ Mengen in FEN-Schreibweise.
- ``s_n_sequence_short``: Alle $S_n$ Mengen in verkürzter FEN-Schreibweise.

In der Datei ``stockfish_compare.txt`` wird nach den Berechnungen festgehalten, zu wie viel % Stockfish und zu wie viel % die selbstgeschriebene KI bessere Ergebnisse erzielt hat. Außerdem wird noch notiert zu welchem Teil sowohl Stockfish, als auch die KI gleiche
Ergebnisse erzielt haben. Zum Nachvollziehen wird dem Nutzer in der Konsole der aktuelle Rechenstand angezeigt.

In [16]:
def compare_sequence_stockfish(sequence_index, s_n_sequence_fen, s_n_sequence_short):
    s_n = s_n_sequence_fen[sequence_index]
    move_count_list = []
    diff_list = []
    length = len(s_n)
    r = 0
    for board_fen in s_n:
        seq_index = sequence_index
        cmp_board = chess.Board(board_fen)
        move_count = 0
        if board_fen in s_n:
            cur_fen = board_fen
            while seq_index > 0:
                board_to_use = chess.Board(cur_fen)
                if board_to_use.turn:
                    # White
                    for p_move in board_to_use.legal_moves:
                        board_to_use.push(p_move)
                        tmp_fen = board_to_use.fen()
                        s_tmp = find_fen_in_sequence(tmp_fen, [s_n_sequence_short[seq_index-1]])
                        if s_tmp != -1:
                            seq_index -= 1
                            move_count += 1
                            cur_fen = board_to_use.fen()
                            break
                        board_to_use.pop()
                else:
                    # Black
                    STOCKFISH.set_fen_position(board_to_use.fen())
                    nxt_move = chess.Move.from_uci(STOCKFISH.get_best_move())
                    board_to_use.push(nxt_move)
                    move_count += 1
                    tmp_fen = board_to_use.fen()
                    seq_index = find_fen_in_sequence(tmp_fen, s_n_sequence_short[:seq_index])
                    cur_fen = board_to_use.fen()
        else:
            print("Couldn't find board!")

        cmp_move_count = 0
        while not cmp_board.is_game_over():
            STOCKFISH.set_fen_position(cmp_board.fen())
            nxt_move = chess.Move.from_uci(STOCKFISH.get_best_move())
            cmp_board.push(nxt_move)
            cmp_move_count += 1
        move_count_list.append((move_count, cmp_move_count, move_count - cmp_move_count))
        diff_list.append(move_count - cmp_move_count)
        clear_output()
        print("Comparing S_" + str(sequence_index) + ":")
        print("Compared " + str(r+1) + "/" + str(length))
        r+=1
    ki_better = 0
    stockfish_better = 0
    equal = 0
    for value in diff_list:
        if value == 0:
            equal += 1
        elif value < 0:
            ki_better += 1
        else:
            stockfish_better += 1
    cmp_values = len(diff_list)
    f = open("Stockfish_compare.txt", "a+")
    f.write("S_" + str(sequence_index) + ":\n")
    f.write("Stockfish war zu " + str(round((stockfish_better/cmp_values) * 100, 2)) + "% besser\n")
    f.write("Die KI war zu " + str(round((ki_better/cmp_values) * 100, 2)) + "% besser\n")
    f.write("Stockfish und die KI haben zu " + str(round((equal/cmp_values) * 100, 2)) + "% die gleichen Ergebnisse erzielt\n")
    f.close()

Als Beispiel wird hier der Vergleich zwischen der KI und Stockfish für die Menge S_20 vorgenommen.

In [18]:
compare_sequence_stockfish(20, S_N_Sequence_fen, S_N_Sequence_fen_short)

Comparing S_0:
Compared 216/216


Ein Testszenario sieht auch vor alle zuvor bestimmten Boards mit dem Lösungsweg von Stockfish zu vergleichen. Hierfür
geht die Funktion `compare_all_sequences` alle S_n-Mengen durch und vergleicht diese jeweils mit der Lösung von
Stockfish.

In [ ]:
def compare_all_sequences(s_n_sequence_fen, s_n_short):
    for i in range(len(s_n_sequence_fen)):
        print("Comparing S_" + str(i) + "...")
        compare_sequence_stockfish(i, s_n_sequence_fen, s_n_short)

In [ ]:
compare_all_sequences(S_N_Sequence_fen, S_N_Sequence_fen_short)

Für die visuelle Darstellung der Züge wurde die Funktion ``show_movelist`` definiert. Diese zeigt auf einem ``chess.Board`` eine mitgegebene Liste von Zügen. Die Funktionsargumente der Funktion lauten:
* ``fen``: Die Fen des Boardes, für welches alle Züge dargestellt werden sollen.
* ``moves``: Die Liste der Spielzüge, welche dargestellt werden sollen.

In [ ]:
def show_movelist(fen, moves):
    presentation_board = chess.Board(fen)
    display(presentation_board)
    time.sleep(2)
    for move in moves:
        presentation_board.push(move)
        clear_output(wait=True)
        display(presentation_board)
        time.sleep(2)

Der Vergleich zwischen der KI und Stockfish kann nun anhand der Länge der Listen durchgeführt werden. Außerdem können die Züge durch die Funktion ``show_movelist`` nachvollzogen werden.

In [ ]:
Moves = calculate_all_moves(FEN, S_N_Sequence_fen_short)
Stockfish_Moves = stockfish_movelist(FEN)

if Moves is not None:
    print("AI needed " + str(len(Moves)) + " moves to beat Stockfish as Black.")
else:
    print("AI found no way to beat Black.")

print("Stockfish needed " + str(len(Stockfish_Moves)) + " moves to win against itself.")

In [ ]:
show_movelist(FEN, Moves)